In [1]:
import tensorflow as tf
# Suppress the annoying tensorflow 1.x deprecation warnings; these make console output
# impossible to parse.
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
# initialize logger to view logging output from segmentation tasks
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2

DEBUG:matplotlib:CACHEDIR=/Users/jpgard/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/jpgard/.matplotlib/fontlist-v300.json


In [4]:
config = '''image {
  hdf5: "data/test/507727402/507727402_raw.h5:raw"
}
image_mean: 78
image_stddev: 20
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "training-logs/lr0.001/model.ckpt-1000000"
model_name: "fftracer.training.models.model.FFNTracerModel"
model_args: "{\\"depth\\": 9, \\"fov_size\\": [49, 49, 1], \\"deltas\\": [8, 8, 0]}"
segmentation_output_dir: "results/mozak"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.1
  min_boundary_dist { x: 1 y: 1 z: 0}
  segment_threshold: 0.01
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [ ]:
### EXPERIMENTAL: show the input image data to verify it is valid
import skimage
from matplotlib import pyplot as plt
skimage.io.imshow(canvas.image[0,:,:])
plt.show()
### END

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1759f6c50>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1759f6c50>
DEBUG:matplotlib.colorbar:Setting pcolormesh
DEBUG:matplotlib.font_manager:findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.
DEBUG:matplotlib.axes._base:update_title_pos
DEBUG:matplotlib.axes._base:update_title_pos


<Figure size 640x480 with 2 Axes>

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
# Create a single segment, starting from the specified origin point.
canvas.segment_at(
    (0, 4521, 3817),  # seed starting point in (z, y, x)
    dynamic_image=inference.DynamicImage(),
    vis_update_every=10,
    vis_fixed_z=True
)

INFO:root:[JG] current pos: (0, 4372, 3701)
INFO:root:[JG] num_iters: 1321
INFO:root:[JG] current pos: (0, 4363, 3721)
INFO:root:[JG] num_iters: 1322
INFO:root:[JG] current pos: (0, 4362, 3704)
INFO:root:[JG] num_iters: 1323
INFO:root:[JG] current pos: (0, 4356, 3719)
INFO:root:[JG] num_iters: 1324
INFO:root:[JG] current pos: (0, 4358, 3752)
INFO:root:[JG] num_iters: 1325
INFO:root:[JG] current pos: (0, 4359, 3761)
INFO:root:[JG] num_iters: 1326
INFO:root:[JG] current pos: (0, 4355, 3736)
INFO:root:[JG] num_iters: 1327
INFO:root:[JG] current pos: (0, 4358, 3731)
INFO:root:[JG] num_iters: 1328
INFO:root:[JG] current pos: (0, 4507, 3982)
INFO:root:[JG] num_iters: 1329
INFO:root:[JG] current pos: (0, 4476, 3981)
INFO:root:[JG] num_iters: 1330


In [ ]:
# read the results
import numpy as np
seg_prob = np.load("results/mozak/0/0/seg-0_0_0.prob")['qprob']
print("shape: {}".format(seg_prob.shape))
seg_prob